In [1]:
import pandas as pd
import yfinance as yf
import datetime
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

# 데이터 불러오기
start = datetime.datetime(2020, 1, 1)
end = datetime.datetime(2024, 12, 27)
btc_data = yf.download('BTC-USD', start=start, end=end)

# 필요한 컬럼 선택
btc_data = btc_data[['Close']]

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [2]:
# 데이터 정규화
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(btc_data)

# 학습 데이터 생성 함수
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 60
X, y = create_dataset(scaled_data, time_step)

In [3]:
# 데이터 차원 변환 [samples, time steps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

In [4]:
# 모델 생성 (최적 파라미터 적용)
model = Sequential([
    LSTM(200, return_sequences=True, input_shape=(time_step, 1)),
    Dropout(0.2),
    LSTM(200, return_sequences=False),
    Dropout(0.2),
    Dense(1)
])

# 모델 컴파일
model.compile(optimizer=Adam(), loss='mean_squared_error')

# 모델 학습
model.fit(X, y, epochs=50, batch_size=32, verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0271
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0013
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0011
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 9.4178e-04
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0010
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 8.8850e-04
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 8.0946e-04
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0018
Epoch 9/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 9.1057e-04
Epoch 10/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 8.0250e-04
Epoch 11/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 8.1599e-04
Epoch 12/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0010
Epoch 13/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 8.1730e-04
Epoch 14/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0010
Epoch 15/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step -

In [5]:
# 예측 데이터 생성
train_size = int(len(scaled_data) * 0.8)
test_data = scaled_data[train_size - time_step:]

X_test, y_test = create_dataset(test_data, time_step)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# 예측
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

# 예측 결과 저장
results_df = pd.DataFrame({'datetime': btc_data.index[-len(predictions):], 'prediction': predictions.flatten()})
results_df.to_csv("lstm_predictions.csv", index=False)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


In [6]:
# 실제 Close 가격과 예측값을 함께 저장하기
btc_data = btc_data[['Close']]
df = pd.read_csv("lstm_predictions.csv")
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)

# 예측값과 실제 Close 가격을 합친 데이터프레임 만들기
btc_data = btc_data.loc[df.index]  # 예측값과 일치하는 날짜의 실제 Close 가격
df['actual_close'] = btc_data['Close']

# DataFrame을 다시 저장.
df.to_csv("lstm_predictions_with_actual.csv", index=True)

In [7]:
pip install backtrader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 26.1 MB/s eta 0:00:00


In [8]:
#LSTM 예측값을 Backtrader에서 사용할 수 있도록
import backtrader as bt
import pandas as pd

# LSTM 예측값 불러오기
df = pd.read_csv("lstm_predictions_with_actual.csv")
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)

# Backtrader용 데이터 피드 생성
class LSTMPredictFeed(bt.feeds.PandasData):
    lines = ('prediction', 'actual_close')  # 예측값과 실제 가격 추가
    params = (
        ('prediction', -1),  # 예측값 컬럼
        ('actual_close', -1)  # 실제 가격 컬럼
    )

In [54]:
class LSTMPredictStrategy(bt.Strategy):
    params = dict(order_size_buy=0.1, order_size_sell=0.2)  # 매수 0.1 코인, 매도 0.2 코인

    def __init__(self):
        self.order = None  # 주문을 추적할 변수

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f"{dt}: {txt}")  # 로그 출력

    def next(self):
        current_price = self.data.actual_close[0]
        predicted_price = self.data.prediction[0]

        # NaN 방지
        if not current_price or not predicted_price or current_price != current_price or predicted_price != predicted_price:
            self.log("❌ NaN 발생 → 거래 없음")
            return

        self.log(f"현재 가격: {current_price}, 예측 가격: {predicted_price}")

        # ✅ 예측 상승 → 0.1 코인 매수
        if predicted_price > current_price:
            cash_available = self.broker.get_cash()
            size = min(self.p.order_size_buy, cash_available / current_price)  # 잔고 고려
            if size > 0:
                self.log(f"✅ 매수 주문 실행 (수량: {size})")
                self.buy(size=size)  # 매수 주문 실행
                self.log(f"현재 잔고: {self.broker.get_cash()}")  # 잔고 확인
                self.log(f"현재 포지션 크기: {self.position.size}")  # 포지션 확인

        # ✅ 예측 하락 → 0.2 코인 매도 (보유 수량이 있을 경우에만)
        elif predicted_price < current_price:
            if self.position.size >= self.p.order_size_sell:
                self.log(f"🔻 매도 주문 실행 (수량: {self.p.order_size_sell})")
                self.sell(size=self.p.order_size_sell)  # 주문 객체 저장
                self.log(f"현재 잔고: {self.broker.get_cash()}")  # 잔고 확인
                self.log(f"현재 포지션 크기: {self.position.size}")  # 포지션 확인
            else:
                self.log("🚫 보유 수량 부족 → 매도 취소")

        # 포트폴리오 가치 수동 계산
        cash = self.broker.get_cash()  # 현재 잔고
        value_of_coins = self.position.size * current_price if self.position else 0  # 보유 코인 가치
        portfolio_value = cash + value_of_coins  # 전체 포트폴리오 가치

        self.log(f"📈 Portfolio Value (Manual Calculation): {portfolio_value}")
        print(f"Portfolio Value (Manual): {portfolio_value}")

In [55]:
#Backtrader 백테스팅 실행
# Cerebro 엔진 생성
cerebro = bt.Cerebro()
cerebro.broker.set_cash(1000000)  # 초기 자본금
# 데이터 로드
data = LSTMPredictFeed(dataname=df)
cerebro.adddata(data)
# 전략 추가
cerebro.addstrategy(LSTMPredictStrategy)
# 수수료 설정 (예: 0.1%)
cerebro.broker.setcommission(commission=0.001)
#전력 실행 전 최초 자산
print("Starting Portfolio Value:", cerebro.broker.getvalue())
# 백테스트 실행
cerebro.run()
# 최종 포트폴리오 값 계산
print("Final Portfolio Value:", cerebro.broker.getvalue())

Starting Portfolio Value: 1000000
2023-12-29: 현재 가격: 42099.40234375, 예측 가격: 43125.434
2023-12-29: ✅ 매수 주문 실행 (수량: 0.1)
2023-12-29: 현재 잔고: 1000000.0
2023-12-29: 현재 포지션 크기: 0
2023-12-29: 📈 Portfolio Value (Manual Calculation): 1000000.0
Portfolio Value (Manual): 1000000.0
2023-12-30: 현재 가격: 42156.90234375, 예측 가격: 42919.42
2023-12-30: ✅ 매수 주문 실행 (수량: 0.1)
2023-12-30: 현재 잔고: 1000000.0
2023-12-30: 현재 포지션 크기: 0
2023-12-30: 📈 Portfolio Value (Manual Calculation): 1000000.0
Portfolio Value (Manual): 1000000.0
2023-12-31: 현재 가격: 42265.1875, 예측 가격: 42496.098
2023-12-31: ✅ 매수 주문 실행 (수량: 0.1)
2023-12-31: 현재 잔고: 1000000.0
2023-12-31: 현재 포지션 크기: 0
2023-12-31: 📈 Portfolio Value (Manual Calculation): 1000000.0
Portfolio Value (Manual): 1000000.0
2024-01-01: 현재 가격: 44167.33203125, 예측 가격: 42256.66
2024-01-01: 🚫 보유 수량 부족 → 매도 취소
2024-01-01: 📈 Portfolio Value (Manual Calculation): 1000000.0
Portfolio Value (Manual): 1000000.0
2024-01-02: 현재 가격: 44957.96875, 예측 가격: 42228.293
2024-01-02: 🚫 보유 수량 부족 → 매도 취소


In [56]:
print("Final Cash:", cerebro.broker.get_cash())
print("Final Portfolio Value:", cerebro.broker.getvalue())

Final Cash: 1000000.0
Final Portfolio Value: nan
